This notebook implements Feature extrcator, such that it looks for the "Node Type", "Connecting Links" and "Layer Type" in an ONNX model.

In [5]:
import onnx
import numpy as np

def extract_features_from_model(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)

    # Extract the graph from the model
    graph = model.graph

    # Initialize lists to store the identified features
    skip_connections = []
    parallel_connections = []
    conv_1x1_3x3_1x1_sets = []

    # Iterate over the nodes in the graph
    for node in graph.node:
        # Identify skip connections
        if len(node.input) > 1:
            skip_connections.append((node.input, node.output))

        # Identify parallel connections
        if len(node.output) > 1:
            parallel_connections.append((node.input, node.output))

        # Identify sets of 1x1, 3x3, and 1x1 convolution layers
        if node.op_type == 'Conv':
            # Get the shape of the convolution kernel
            kernel_shape = node.attribute[1].ints

            # Check if it's a 1x1, 3x3, or 1x1 convolution
            if kernel_shape == [1, 1]:
                conv_1x1_3x3_1x1_sets.append('1x1 Convolution')
            elif kernel_shape == [3, 3]:
                conv_1x1_3x3_1x1_sets.append('3x3 Convolution')
            elif kernel_shape == [1, 1]:
                conv_1x1_3x3_1x1_sets.append('1x1 Convolution')

    # Print the identified features
    print("Skip Connections:")
    for skip_conn in skip_connections:
        print(skip_conn)
    
    print("\nParallel Connections:")
    for parallel_conn in parallel_connections:
        print(parallel_conn)

    print("\nSets of 1x1, 3x3, and 1x1 Convolution Layers:")
    for conv_set in conv_1x1_3x3_1x1_sets:
        print(conv_set)

# Example usage
model_path = "ONNX_patterns/art_block1.onnx"
extract_features_from_model(model_path)


Skip Connections:
(['/layers/layers.0/layers.0.0/relu_2/Relu_output_0', 'onnx::Conv_993', 'onnx::Conv_994'], ['/layers/layers.0/layers.0.1/conv1/Conv_output_0'])
(['/layers/layers.0/layers.0.1/relu/Relu_output_0', 'onnx::Conv_996', 'onnx::Conv_997'], ['/layers/layers.0/layers.0.1/conv2/Conv_output_0'])
(['/layers/layers.0/layers.0.1/relu_1/Relu_output_0', 'onnx::Conv_999', 'onnx::Conv_1000'], ['/layers/layers.0/layers.0.1/conv3/Conv_output_0'])
(['/layers/layers.0/layers.0.1/conv3/Conv_output_0', '/layers/layers.0/layers.0.0/relu_2/Relu_output_0'], ['/layers/layers.0/layers.0.1/Add_output_0'])
(['/layers/layers.0/layers.0.1/relu_2/Relu_output_0', 'onnx::Conv_1002', 'onnx::Conv_1003'], ['/layers/layers.0/layers.0.2/conv1/Conv_output_0'])
(['/layers/layers.0/layers.0.2/relu/Relu_output_0', 'onnx::Conv_1005', 'onnx::Conv_1006'], ['/layers/layers.0/layers.0.2/conv2/Conv_output_0'])
(['/layers/layers.0/layers.0.2/relu_1/Relu_output_0', 'onnx::Conv_1008', 'onnx::Conv_1009'], ['/layers/layers

In [4]:
import onnx

def identify_skip_connections(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)

    # Extract the graph from the model
    graph = model.graph

    # Initialize a list to store skip connections
    skip_connections = []

    # Create a dictionary to map output names to nodes
    node_map = {node.name: node for node in graph.node}

    # Iterate over the nodes in the graph
    for node in graph.node:
        # Identify skip connections
        if len(node.input) > 1:
            input_node_name = node.input[0]
            if input_node_name in node_map:
                input_node = node_map[input_node_name]
                if len(input_node.output) > 1:
                    skip_connections.append((input_node_name, node.name))

    # Print the identified skip connections
    if len(skip_connections) > 0:
        print("Skip Connections:")
        for skip_conn in skip_connections:
            print(skip_conn)
    else:
        print("No skip connections found in the model.")

# Example usage
model_path = "ONNX_models/vgg16-7.onnx" 
identify_skip_connections(model_path)

model_path = "ONNX_patterns/Resnet.onnx"
identify_skip_connections(model_path)

model_path = "ONNX_models/resnet18-v1-7.onnx"
identify_skip_connections(model_path)


No skip connections found in the model.
No skip connections found in the model.
No skip connections found in the model.


In [1]:
### Feature extractor##
#Reading ONNX model to look for features#

In [9]:
#Split, Clip, Concat, Add node/layer

import onnx

def check_node_type(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)

    # Extract the graph from the model
    graph = model.graph

    # Check if the model has any split layers
    has_split_layer = False
    has_clip_layer = False
    has_concat_layer = False
    has_add_layer = False

    # Iterate over the nodes in the graph
    for node in graph.node:
        # Check if the node is a Split Node and similary other nodes Clip, Concat and Add nodes
        if node.op_type == 'Split':
            has_split_layer = True
            break

        if node.op_type == 'Clip':
            has_clip_layer = True
            break
        if node.op_type == 'Concat':
            has_concat_layer = True
            break

        if node.op_type == 'Add':
            has_add_layer = True
            break

    # Print the result
    if has_concat_layer:
        print("The model has a Concatenation layer.")
    else:
        print("The model does not have a Concatenation layer.")


    # Print the result
    if has_split_layer:
        print("The model has a Split layer.")
    else:
        print("The model does not have a Split layer.")

    # Print the result
    if has_clip_layer:
        print("The model has a Clip layer.")
    else:
        print("The model does not have a Clip layer.")
 

    # Print the result
    if has_add_layer:
        print("The model has a Add layer.")
    else:
        print("The model does not have a Add layer.")

model_path = 'ONNX_patterns/Shufflenet-1.onnx'
check_node_type(model_path)

model_path = 'ONNX_patterns/Denseblock.onnx'
check_node_type(model_path)

model_path = 'ONNX_patterns/resnest.onnx'
check_node_type(model_path)

model_path = 'ONNX_patterns/ESPNet-1.onnx'
check_node_type(model_path)

model_path = 'ONNX_patterns/Inception-g.onnx'
check_node_type(model_path)

model_path = 'ONNX_patterns/Resnet.onnx'
check_node_type(model_path)

model_path = 'ONNX_patterns/art_block1.onnx'
check_node_type(model_path)

model_path = 'ONNX_patterns/squeezeNet-1.onnx'
check_node_type(model_path)

model_path = 'ONNX_patterns/bottleneck_residual.onnx'
check_node_type(model_path)


The model has a Split layer.
The model does not have a Split layer.
The model has a Split layer.
The model does not have a Split layer.
The model does not have a Split layer.
The model does not have a Split layer.
The model does not have a Split layer.
The model does not have a Split layer.
The model does not have a Split layer.


In [14]:
#Parallel connections CHECK this again

import onnx

def check_parallel_connections(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)

    # Extract the graph from the model
    graph = model.graph

    # Initialize a flag to track parallel connections
    has_parallel_connections = False

    # Iterate over the nodes in the graph
    for node in graph.node:
        # Check if the node has multiple outputs
        if len(node.output) > 1:
            has_parallel_connections = True
            break

    # Print the result
    if has_parallel_connections:
        print("The model has parallel connections.")
    else:
        print("The model does not have parallel connections.")

# Example usage



model_path = 'ONNX_patterns/Denseblock.onnx'  
check_parallel_connections(model_path)

model_path = 'ONNX_patterns/resnest.onnx'
check_parallel_connections(model_path)

model_path = 'ONNX_patterns/resnest1.onnx'
check_parallel_connections(model_path)

model_path = 'ONNX_patterns/ESPNet-1.onnx'
check_parallel_connections(model_path)


model_path = 'ONNX_patterns/Resnet.onnx' 
check_parallel_connections(model_path)

model_path = 'ONNX_patterns/art_block1.onnx'
check_parallel_connections(model_path)

model_path = 'ONNX_patterns/squeeze_and_excite-3.onnx'
check_parallel_connections(model_path)


model_path = 'ONNX_patterns/bottleneck_residual.onnx'
check_parallel_connections(model_path)

The model has parallel connections.
The model does not have parallel connections.
The model has parallel connections.
The model has parallel connections.
The model does not have parallel connections.
The model does not have parallel connections.
The model does not have parallel connections.
The model does not have parallel connections.
The model does not have parallel connections.
The model does not have parallel connections.
The model does not have parallel connections.


In [18]:
#Dense block

import onnx

def check_dense_block(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)

    # Extract the graph from the model
    graph = model.graph

    # Initialize a flag to track the presence of a dense block
    has_dense_block = False

    # Initialize a dictionary to store output shapes of nodes
    node_shapes = {}

    # Iterate over the nodes in the graph to get output shape information
    for node in graph.node:
        # Iterate over the outputs of the node and store their shape information
        for output in node.output:
            output_shape = None
            for value_info in graph.value_info:
                if value_info.name == output:
                    output_shape = value_info.type.tensor_type.shape
                    break
            if output_shape is None:
                for value_info in graph.output:
                    if value_info.name == output:
                        output_shape = value_info.type.tensor_type.shape
                        break
            if output_shape is not None:
                node_shapes[output] = output_shape

    # Iterate over the nodes again to check for dense block pattern
    for node in graph.node:
        # Check if the node is a Convolution operation
        if node.op_type == 'Conv':
            # Get the input shape of the Convolution operation
            input_shape = None
            for input_name in node.input:
                if input_name in node_shapes:
                    input_shape = node_shapes[input_name]
                    break

            # Check if the input shape has a dimension with value greater than 1
            if input_shape is not None:
                for dim in input_shape.dim:
                    if dim.dim_value > 1:
                        has_dense_block = True
                        break

        # Break out of the loop if a dense block is found
        if has_dense_block:
            break

    # Print the result
    if has_dense_block:
        print("The model has a dense block.")
    else:
        print("The model does not have a dense block.")

#Wrong result - - Not workingg.

model_path = 'ONNX_patterns/Shufflenet-1.onnx' 
check_dense_block(model_path)
model_path = 'ONNX_patterns/resnest.onnx'
check_dense_block(model_path)
model_path = 'ONNX_patterns/resnest1.onnx'
check_dense_block(model_path)
model_path = 'ONNX_patterns/ESPNet-1.onnx'
check_dense_block(model_path)
model_path = 'ONNX_patterns/Inception-g.onnx' 
check_dense_block(model_path)


The model does not have a dense block.
The model does not have a dense block.


The model does not have a dense block.
The model does not have a dense block.
The model does not have a dense block.
The model does not have a dense block.
The model does not have a dense block.
The model does not have a dense block.
The model does not have a dense block.
The model does not have a dense block.
The model does not have a dense block.
The model does not have a dense block.


In [19]:
#Fixed conv size

import onnx

def check_fixed_kernel_size(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)

    # Extract the graph from the model
    graph = model.graph

    # Iterate over the nodes in the graph
    for node in graph.node:
        # Check if the node is a Convolution operation
        if node.op_type == 'Conv':
            # Get the kernel shape of the Convolution operation
            kernel_shape = None
            for attr in node.attribute:
                if attr.name == 'kernel_shape':
                    kernel_shape = attr.ints
                    break

            # Check if the kernel shape is fixed
            if kernel_shape is not None:
                is_fixed_kernel_size = all(dim == kernel_shape[0] for dim in kernel_shape)
                if is_fixed_kernel_size:
                    print("Convolution layer with fixed kernel size:", kernel_shape)
                else:
                    print("Convolution layer with variable kernel size:", kernel_shape)



model_path = 'ONNX_patterns/Shufflenet-1.onnx' 
check_fixed_kernel_size(model_path)
model_path = 'ONNX_patterns/Denseblock.onnx'  
check_fixed_kernel_size(model_path)
model_path = 'ONNX_patterns/resnest.onnx'
check_fixed_kernel_size(model_path)
model_path = 'ONNX_patterns/resnest1.onnx'
check_fixed_kernel_size(model_path)
model_path = 'ONNX_patterns/ESPNet-1.onnx'
check_fixed_kernel_size(model_path)
model_path = 'ONNX_patterns/Inception-g.onnx' 
check_fixed_kernel_size(model_path)

model_path = 'ONNX_patterns/Resnet.onnx' 
check_fixed_kernel_size(model_path)
model_path = 'ONNX_patterns/art_block1.onnx'
check_fixed_kernel_size(model_path)
model_path = 'ONNX_patterns/squeeze_and_excite-3.onnx'
check_fixed_kernel_size(model_path)
model_path = 'ONNX_patterns/squeezeNet-1.onnx'  
check_fixed_kernel_size(model_path)
model_path = 'ONNX_patterns/bottleneck_residual.onnx'
check_fixed_kernel_size(model_path)


Convolution layer with fixed kernel size: [1, 1]
Convolution layer with fixed kernel size: [3, 3]
Convolution layer with fixed kernel size: [1, 1]
Convolution layer with fixed kernel size: [1, 1]
Convolution layer with fixed kernel size: [3, 3]
Convolution layer with fixed kernel size: [1, 1]
Convolution layer with fixed kernel size: [3, 3]
Convolution layer with fixed kernel size: [1, 1]
Convolution layer with fixed kernel size: [3, 3]
Convolution layer with fixed kernel size: [1, 1]
Convolution layer with fixed kernel size: [3, 3]
Convolution layer with fixed kernel size: [3, 3]
Convolution layer with fixed kernel size: [3, 3]
Convolution layer with fixed kernel size: [3, 3]
Convolution layer with fixed kernel size: [1, 1]
Convolution layer with fixed kernel size: [3, 3]
Convolution layer with fixed kernel size: [1, 1]
Convolution layer with fixed kernel size: [1, 1]
Convolution layer with fixed kernel size: [1, 1]
Convolution layer with fixed kernel size: [1, 1]
Convolution layer wi

In [3]:
#Bottleneck layers
import onnx

def check_kernel_sequence(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)

    # Extract the graph from the model
    graph = model.graph

    # Initialize a flag for checking the kernel sequence
    has_kernel_sequence = False

    # Iterate over the nodes in the graph
    for i in range(len(graph.node) - 2):
        node1 = graph.node[i]
        node2 = graph.node[i + 1]
        node3 = graph.node[i + 2]

        # Check if the nodes are Convolution operations
        if (
            node1.op_type == 'Conv'
            and node2.op_type == 'Conv'
            and node3.op_type == 'Conv'
        ):
            # Get the kernel shapes of the Convolution operations
            kernel_shape1 = None
            kernel_shape2 = None
            kernel_shape3 = None
            for attr in node1.attribute:
                if attr.name == 'kernel_shape':
                    kernel_shape1 = attr.ints
                    break
            for attr in node2.attribute:
                if attr.name == 'kernel_shape':
                    kernel_shape2 = attr.ints
                    break
            for attr in node3.attribute:
                if attr.name == 'kernel_shape':
                    kernel_shape3 = attr.ints
                    break

            # Check if the kernel shapes form the sequence 1x1, 3x3, 1x1
            if (
                kernel_shape1 == [1, 1]
                and kernel_shape2 == [3, 3]
                and kernel_shape3 == [1, 1]
            ):
                has_kernel_sequence = True
                break

    if has_kernel_sequence:
        print("The model has a convolution layer sequence of 1x1, 3x3, 1x1.")
    else:
        print("The model does not have a convolution layer sequence of 1x1, 3x3, 1x1.")


#Wrong output again.

model_path = 'ONNX_patterns/Shufflenet-1.onnx' 
check_kernel_sequence(model_path)
model_path = 'ONNX_patterns/Denseblock.onnx'  
check_kernel_sequence(model_path)

model_path = 'ONNX_patterns/resnest.onnx'
check_kernel_sequence(model_path)
model_path = 'ONNX_patterns/resnest1.onnx'
check_kernel_sequence(model_path)
model_path = 'ONNX_patterns/ESPNet-1.onnx'
check_kernel_sequence(model_path)


The model does not have a convolution layer sequence of 1x1, 3x3, 1x1.
The model does not have a convolution layer sequence of 1x1, 3x3, 1x1.
The model does not have a convolution layer sequence of 1x1, 3x3, 1x1.
The model does not have a convolution layer sequence of 1x1, 3x3, 1x1.
The model does not have a convolution layer sequence of 1x1, 3x3, 1x1.
The model does not have a convolution layer sequence of 1x1, 3x3, 1x1.
The model does not have a convolution layer sequence of 1x1, 3x3, 1x1.
The model does not have a convolution layer sequence of 1x1, 3x3, 1x1.
The model does not have a convolution layer sequence of 1x1, 3x3, 1x1.
The model does not have a convolution layer sequence of 1x1, 3x3, 1x1.
The model does not have a convolution layer sequence of 1x1, 3x3, 1x1.
The model does not have a convolution layer sequence of 1x1, 3x3, 1x1.
The model does not have a convolution layer sequence of 1x1, 3x3, 1x1.
The model does not have a convolution layer sequence of 1x1, 3x3, 1x1.


In [2]:
import onnx

def check_model_features(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)

    # Extract the graph from the model
    graph = model.graph

    # Initialize flags for features
    has_skip_connections = False
    has_parallel_connections = False
    has_conv_1x1_3x3_1x1_sets = False

    # Iterate over the nodes in the graph
    for node in graph.node:
        # Check for skip connections
        if len(node.input) > 1:
            has_skip_connections = True

        # Check for parallel connections
        if len(node.output) > 1:
            has_parallel_connections = True

        # Check for sets of 1x1, 3x3, and 1x1 convolution layers
        if node.op_type == 'Conv':
            # Get the shape of the convolution kernel
            kernel_shape = node.attribute[1].ints

            # Check if it's a 1x1, 3x3, or 1x1 convolution
            if kernel_shape == [1, 1]:
                has_conv_1x1_3x3_1x1_sets = True
            elif kernel_shape == [3, 3]:
                has_conv_1x1_3x3_1x1_sets = True
            elif kernel_shape == [1, 1]:
                has_conv_1x1_3x3_1x1_sets = True

    # Print the result
    print("Skip Connections:", "Yes" if has_skip_connections else "No")
    print("Parallel Connections:", "Yes" if has_parallel_connections else "No")
    print("Sets of 1x1, 3x3, and 1x1 Convolution Layers:", "Yes" if has_conv_1x1_3x3_1x1_sets else "No")

# Example usage
model_path = "ONNX_patterns/art_block1.onnx"  
check_model_features(model_path)


Skip Connections: Yes
Parallel Connections: No
Sets of 1x1, 3x3, and 1x1 Convolution Layers: No


In [3]:
model_path = "ONNX_patterns/Inception-g.onnx"  
check_model_features(model_path)


Skip Connections: Yes
Parallel Connections: No
Sets of 1x1, 3x3, and 1x1 Convolution Layers: No


In [4]:
model_path = "ONNX_patterns/Denseblock.onnx"  
check_model_features(model_path)

model_path = "ONNX_patterns/ESPNet-1.onnx"  
check_model_features(model_path)

model_path = "ONNX_patterns/Resnet.onnx"  
check_model_features(model_path)


Skip Connections: Yes
Parallel Connections: No
Sets of 1x1, 3x3, and 1x1 Convolution Layers: No
Skip Connections: Yes
Parallel Connections: No
Sets of 1x1, 3x3, and 1x1 Convolution Layers: No
Skip Connections: Yes
Parallel Connections: No
Sets of 1x1, 3x3, and 1x1 Convolution Layers: No


In [6]:
model_path = "ONNX_models/resnet18-v1-7.onnx"  
check_model_features(model_path)

model_path = "ONNX_models/densenet-3.onnx"  
check_model_features(model_path)

model_path = "ONNX_models/espnetv2_s_2.0_imsize_224x224_imagenet.onnx"  
check_model_features(model_path)

model_path = "ONNX_models/resnext.onnx"  
check_model_features(model_path)


Skip Connections: Yes
Parallel Connections: No
Sets of 1x1, 3x3, and 1x1 Convolution Layers: No
Skip Connections: Yes
Parallel Connections: No
Sets of 1x1, 3x3, and 1x1 Convolution Layers: No
Skip Connections: Yes
Parallel Connections: Yes
Sets of 1x1, 3x3, and 1x1 Convolution Layers: No
Skip Connections: Yes
Parallel Connections: No
Sets of 1x1, 3x3, and 1x1 Convolution Layers: No


In [18]:
import onnx

def check_convolution_layers(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)

    # Extract the graph from the model
    graph = model.graph

    # Initialize flags for the presence of 1x1 and 3x3 convolution layers
    has_1x1_layer = False
    has_3x3_layer = False

    # Iterate over the nodes in the graph
    for node in graph.node:
        if node.op_type == 'Conv':
            kernel_shape = node.attribute[1].ints

            if kernel_shape == [1, 1]:
                has_1x1_layer = True
            elif kernel_shape == [3, 3]:
                has_3x3_layer = True

    # Print the result
    print("Presence of 1x1 Convolution Layer:", has_1x1_layer)
    print("Presence of 3x3 Convolution Layer:", has_3x3_layer)

# Example usage


model_path = "ONNX_patterns/Inception-g.onnx"  
check_convolution_layers(model_path)
model_path = "ONNX_patterns/Inception.onnx"
check_convolution_layers(model_path)
model_path = "ONNX_patterns/bottleneck_residual.onnx"  
check_convolution_layers(model_path)
model_path = "ONNX_patterns/Resnet.onnx"
check_convolution_layers(model_path)
model_path = "ONNX_models/inception-v2-3.onnx"
check_convolution_layers(model_path)

Presence of 1x1 Convolution Layer: False
Presence of 3x3 Convolution Layer: False
Presence of 1x1 Convolution Layer: False
Presence of 3x3 Convolution Layer: False
Presence of 1x1 Convolution Layer: False
Presence of 3x3 Convolution Layer: False
Presence of 1x1 Convolution Layer: False
Presence of 3x3 Convolution Layer: False
Presence of 1x1 Convolution Layer: False
Presence of 3x3 Convolution Layer: False


In [10]:
# Only Sequential connection exist in the model/ pattern.

import onnx

def identify_seq_connections(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)

    # Extract the graph from the model
    graph = model.graph

    # Initialize a list to store skip connections
    has_seq_connections = False

    # Create a dictionary to map output names to nodes
    node_map = {node.name: node for node in graph.node}

    # Iterate over the nodes in the graph
    for node in graph.node:
        # Identify sequnetial connections
        if len(node.input) == 1:
            input_node_name = node.input[0]
            if input_node_name in node_map:
                input_node = node_map[input_node_name]
                if len(input_node.output) ==1:
                    has_seq_connections = True

    print('Sequential connections:',has_seq_connections)



# Example usage

model_path = "ONNX_patterns/Resnet.onnx"
identify_seq_connections(model_path)

model_path = "ONNX_models/vgg16-7.onnx" 
identify_seq_connections(model_path)

model_path = "ONNX_patterns/Resnet.onnx"
identify_seq_connections(model_path)

model_path = "ONNX_models/resnet18-v1-7.onnx"
identify_seq_connections(model_path)


model_path = "ONNX_models/densenet-3.onnx"
identify_seq_connections(model_path)

model_path = "ONNX_models/efficientnet-lite4-11.onnx"
identify_seq_connections(model_path)

model_path = "ONNX_patterns/ESPNet-1.onnx"
identify_seq_connections(model_path)

model_path = "ONNX_patterns/Inception-g.onnx"
identify_seq_connections(model_path)

model_path = "ONNX_patterns/Shufflenet-1.onnx"
identify_seq_connections(model_path)

model_path = "ONNX_patterns/art_block1.onnx"
identify_seq_connections(model_path)

model_path = "ONNX_patterns/bottleneck_residual.onnx"
identify_seq_connections(model_path)

model_path = "ONNX_patterns/resnest.onnx"
identify_seq_connections(model_path)

model_path = "ONNX_patterns/squeezeNet-1.onnx"  #Issue with this model / pattern
identify_seq_connections(model_path)

model_path = "ONNX_patterns/squeeze_and_excite-1.onnx"
identify_seq_connections(model_path)


Sequential connections: True
Sequential connections: True
Sequential connections: True
Sequential connections: True
Sequential connections: False
Sequential connections: False
Sequential connections: False
Sequential connections: False
Sequential connections: False
Sequential connections: False
Sequential connections: False
Sequential connections: False
Sequential connections: True
Sequential connections: False


In [12]:
# Skip connection exist in the model/ pattern.

import onnx

def identify_skip_connections(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)

    # Extract the graph from the model
    graph = model.graph

    # Initialize a list to store skip connections
    has_skip_connections = False

    # Create a dictionary to map output names to nodes
    node_map = {node.name: node for node in graph.node}

    # Iterate over the nodes in the graph
    for node in graph.node:
        # Identify skip connections
        if len(node.input) > 1:
            input_node_name = node.input[0]
            if input_node_name in node_map:
                input_node = node_map[input_node_name]
                if len(input_node.output) ==1:
                    has_skip_connections = True

    print('Skip connections:',has_skip_connections)



# Example usage

model_path = "ONNX_patterns/Resnet.onnx"
identify_skip_connections(model_path)

model_path = "ONNX_models/vgg16-7.onnx" #Issue
identify_skip_connections(model_path)

model_path = "ONNX_patterns/Resnet.onnx"
identify_skip_connections(model_path)

model_path = "ONNX_models/resnet18-v1-7.onnx"
identify_skip_connections(model_path)


model_path = "ONNX_models/densenet-3.onnx"
identify_skip_connections(model_path)

model_path = "ONNX_models/efficientnet-lite4-11.onnx"
identify_skip_connections(model_path)

model_path = "ONNX_patterns/ESPNet-1.onnx"
identify_skip_connections(model_path)

model_path = "ONNX_patterns/Inception-g.onnx"
identify_skip_connections(model_path)

model_path = "ONNX_patterns/Shufflenet-1.onnx"
identify_skip_connections(model_path)

model_path = "ONNX_patterns/art_block1.onnx"
identify_skip_connections(model_path)

model_path = "ONNX_patterns/bottleneck_residual.onnx"
identify_skip_connections(model_path)

model_path = "ONNX_patterns/resnest.onnx"
identify_skip_connections(model_path)

model_path = "ONNX_patterns/squeezeNet-1.onnx"  
identify_skip_connections(model_path)

model_path = "ONNX_patterns/squeeze_and_excite-1.onnx"
identify_skip_connections(model_path)


Skip connections: True
Skip connections: True
Skip connections: True
Skip connections: True
Skip connections: False
Skip connections: False
Skip connections: False
Skip connections: False
Skip connections: False
Skip connections: False
Skip connections: False
Skip connections: False
Skip connections: True
Skip connections: False


In [5]:
# Parallel connection exist in the model/ pattern.

import onnx

def identify_parallel_connections(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)

    # Extract the graph from the model
    graph = model.graph

    # Initialize a list to store skip connections
    has_parallel_connections = False


    for node in graph.node:
        # Identify parallel connections
        if len(node.output) > 1:
            has_parallel_connections = True

    print('Parallel connections:',has_parallel_connections)



# Example usage

model_path = "ONNX_patterns/Resnet.onnx"
identify_parallel_connections(model_path)

model_path = "ONNX_models/vgg16-7.onnx"
identify_parallel_connections(model_path)

model_path = "ONNX_patterns/Resnet.onnx"
identify_parallel_connections(model_path)

model_path = "ONNX_models/resnet18-v1-7.onnx"
identify_parallel_connections(model_path)

model_path = "ONNX_models/densenet-3.onnx"
identify_parallel_connections(model_path)

model_path = "ONNX_models/efficientnet-lite4-11.onnx"
identify_parallel_connections(model_path)

model_path = "ONNX_patterns/ESPNet-1.onnx"
identify_parallel_connections(model_path)

model_path = "ONNX_patterns/Inception-g.onnx"
identify_parallel_connections(model_path)

model_path = "ONNX_patterns/Shufflenet-1.onnx"
identify_parallel_connections(model_path)

model_path = "ONNX_patterns/art_block1.onnx"
identify_parallel_connections(model_path)

model_path = "ONNX_patterns/bottleneck_residual.onnx"
identify_parallel_connections(model_path)

model_path = "ONNX_patterns/resnest.onnx"
identify_parallel_connections(model_path)

model_path = "ONNX_patterns/squeezeNet-1.onnx"  
identify_parallel_connections(model_path)

model_path = "ONNX_patterns/squeeze_and_excite-1.onnx"
identify_parallel_connections(model_path)

model_path = "ONNX_patterns/Inception.onnx"
identify_parallel_connections(model_path)

model_path = "ONNX_models/inception-v2-3.onnx"
identify_parallel_connections(model_path)

model_path = "ONNX_models/googlenet-12-int8.onnx"
identify_parallel_connections(model_path)

Parallel connections: False
Parallel connections: False
Parallel connections: False
Parallel connections: False
Parallel connections: False
Parallel connections: False
Parallel connections: False
Parallel connections: False
Parallel connections: True
Parallel connections: False
Parallel connections: False
Parallel connections: True
Parallel connections: False
Parallel connections: False
Parallel connections: False
Parallel connections: False
Parallel connections: False


In [6]:
def identify_dense_connections(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)

    # Extract the graph from the model
    graph = model.graph

    # Initialize a flag to track the presence of dense connections
    has_dense_connections = False

    # Create a dictionary to map output names to nodes
    node_map = {node.name: node for node in graph.node}

    # Iterate over the nodes in the graph
    for node in graph.node:
        # Check if the node is a Concatenation operation
        if node.op_type == 'Concat':
            # Check if the node has multiple inputs
            if len(node.input) > 1:
                # Iterate over the inputs of the Concatenation operation
                for input_name in node.input:
                    if input_name in node_map:
                        input_node = node_map[input_name]
                        # Check if the input node is a Convolution operation
                        if input_node.op_type == 'Conv':
                            has_dense_connections = True
                            break

        # Break out of the loop if dense connections are found
        if has_dense_connections:
            break

    # Print the result
    if has_dense_connections:
        print("The model has dense connections.")
    else:
        print("The model does not have dense connections.")


In [15]:
def identify_dense_connections(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)

    # Extract the graph from the model
    graph = model.graph

    # Create a dictionary to map output names to nodes
    node_map = {node.name: node for node in graph.node}

    # Initialize a flag to track the presence of dense connections
    has_dense_connections = False

    # Iterate over the nodes in the graph
    for node in graph.node:
        # Check if the node is a Concat operation
        if node.op_type == 'Concat':
            # Get the input nodes of the Concat operation
            input_nodes = []
            for input_name in node.input:
                if input_name in node_map:
                    input_node = node_map[input_name]
                    input_nodes.append(input_node)

            # Check if the input nodes are connected to each other
            for i in range(len(input_nodes)):
                for j in range(i+1, len(input_nodes)):
                    if input_nodes[i] in input_nodes[j].input:
                        has_dense_connections = True
                        break

            if has_dense_connections:
                break

    # Print the result
    if has_dense_connections:
        print("The model has dense connections.")
    else:
        print("The model does not have dense connections.")





In [17]:
def detect_conv_sequence(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)

    # Extract the graph from the model
    graph = model.graph

    # Initialize flags for detecting convolution sequences
    has_conv_sequence = False
    prev_kernel_shape = None

    # Iterate over the nodes in the graph
    for node in graph.node:
        # Check if the node is a Conv operation
        if node.op_type == 'Conv':
            # Get the kernel shape of the Convolution operation
            kernel_shape = None
            for attr in node.attribute:
                if attr.name == 'kernel_shape':
                    kernel_shape = attr.ints
                    break

            # If the previous kernel shape is not set or if the current kernel shape is not sequential,
            # reset the flag and previous kernel shape
            if prev_kernel_shape is None or kernel_shape != [3, 3]:
                has_conv_sequence = False
                prev_kernel_shape = None
            elif kernel_shape == [3, 3] and prev_kernel_shape == [1, 1]:
                # Detected a sequential convolutional layer sequence
                has_conv_sequence = True
                break

            # Set the previous kernel shape
            prev_kernel_shape = kernel_shape

    # Print the result
    if has_conv_sequence:
        print("The model has the convolutional layer sequence: 1x1, 3x3, 1x1")
    else:
        print("The model does not have the convolutional layer sequence: 1x1, 3x3, 1x1")


In [19]:
model_path = "ONNX_patterns/Denseblock.onnx"
detect_conv_sequence(model_path)

model_path = "ONNX_patterns/Resnet.onnx"
detect_conv_sequence(model_path)

model_path = "ONNX_patterns/ESPNet-1.onnx"
detect_conv_sequence(model_path)

print("%")

model_path = "ONNX_patterns/Inception-g.onnx"
detect_conv_sequence(model_path)

model_path = "ONNX_patterns/Shufflenet-1.onnx"
detect_conv_sequence(model_path)

model_path = "ONNX_patterns/art_block1.onnx"
detect_conv_sequence(model_path)

print("%&")

model_path = "ONNX_patterns/bottleneck_residual.onnx"
detect_conv_sequence(model_path)

model_path = "ONNX_patterns/resnest.onnx"
detect_conv_sequence(model_path)

model_path = "ONNX_patterns/squeezeNet-1.onnx"  
detect_conv_sequence(model_path)

model_path = "ONNX_patterns/squeeze_and_excite-1.onnx"
detect_conv_sequence(model_path)

model_path = "ONNX_patterns/Inception.onnx"
detect_conv_sequence(model_path)




The model has the convolutional layer sequence: 1x1, 3x3, 1x1
The model does not have the convolutional layer sequence: 1x1, 3x3, 1x1
The model has the convolutional layer sequence: 1x1, 3x3, 1x1
%
The model does not have the convolutional layer sequence: 1x1, 3x3, 1x1
The model has the convolutional layer sequence: 1x1, 3x3, 1x1
The model has the convolutional layer sequence: 1x1, 3x3, 1x1
%&
The model has the convolutional layer sequence: 1x1, 3x3, 1x1
The model has the convolutional layer sequence: 1x1, 3x3, 1x1
The model has the convolutional layer sequence: 1x1, 3x3, 1x1
The model has the convolutional layer sequence: 1x1, 3x3, 1x1
The model has the convolutional layer sequence: 1x1, 3x3, 1x1


In [23]:
def detect_conv5_sequence(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)

    # Extract the graph from the model
    graph = model.graph

    # Initialize flags for detecting convolution sequences
    has_conv_sequence = False
    prev_kernel_shape = None

    # Iterate over the nodes in the graph
    for node in graph.node:
        # Check if the node is a Conv operation
        if node.op_type == 'Conv' or node.op_type == "QLinearConv":
            # Get the kernel shape of the Convolution operation
            kernel_shape = None
            for attr in node.attribute:
                if attr.name == 'kernel_shape':
                    kernel_shape = attr.ints
                    break

            # If the previous kernel shape is not set or if the current kernel shape is not sequential,
            # reset the flag and previous kernel shape
            if prev_kernel_shape is None or kernel_shape not in [[3, 3], [5, 5]]:
                has_conv_sequence = False
                prev_kernel_shape = None
            elif kernel_shape == [3, 3] and prev_kernel_shape == [1, 1]:
                # Detected a sequential convolutional layer sequence [1x1, 3x3]
                # Check if the next layer is 5x5
                next_node = node
                if next_node.output:
                    next_output_name = next_node.output[0]
                    for next_node in graph.node:
                        if next_output_name in next_node.input and (next_node.op_type == 'Conv' or next_node.op_type == "QLinearConv") :
                            next_kernel_shape = None
                            for attr in next_node.attribute:
                                if attr.name == 'kernel_shape':
                                    next_kernel_shape = attr.ints
                                    break
                            if next_kernel_shape == [5, 5]:
                                has_conv_sequence = True
                                break

            # Set the previous kernel shape
            prev_kernel_shape = kernel_shape

    # Print the result
    if has_conv_sequence:
        print("The model has the convolutional layer sequence: 1x1, 3x3, 5x5")
    else:
        print("The model does not have the convolutional layer sequence: 1x1, 3x3, 5x5")


In [25]:
def detect_parallel_conv_sequence(model_path):
    # Load the ONNX model
    model = onnx.load(model_path)

    # Extract the graph from the model
    graph = model.graph

    # Initialize flags for detecting parallel convolution sequences
    has_parallel_conv_sequence = False

    # Iterate over the nodes in the graph
    for node in graph.node:
        # Check if the node is a Conv operation
        if node.op_type == 'Conv' or node.op_type=="QLinearConv":
            # Get the kernel shape of the Convolution operation
            kernel_shape = None
            for attr in node.attribute:
                if attr.name == 'kernel_shape':
                    kernel_shape = attr.ints
                    break

            # Check if the current Convolution operation has the expected kernel shape
            if kernel_shape == [1, 1]:
                # Check if the next Convolution operation is in parallel connection
                next_node = node
                if next_node.output:
                    next_output_name = next_node.output[0]
                    for next_node in graph.node:
                        if next_output_name in next_node.input and (next_node.op_type == 'Conv' or node.op_type=="QLinearConv"):
                            next_kernel_shape = None
                            for attr in next_node.attribute:
                                if attr.name == 'kernel_shape':
                                    next_kernel_shape = attr.ints
                                    break
                            if next_kernel_shape == [3, 3]:
                                # Check if the next Convolution operation is also in parallel connection
                                next_output_name = next_node.output[0]
                                for next_node in graph.node:
                                    if next_output_name in next_node.input and (next_node.op_type == 'Conv' or node.op_type=="QLinearConv"):
                                        next_kernel_shape = None
                                        for attr in next_node.attribute:
                                            if attr.name == 'kernel_shape':
                                                next_kernel_shape = attr.ints
                                                break
                                        if next_kernel_shape == [5, 5]:
                                            # Check if the final Convolution operation is in parallel connection
                                            next_output_name = next_node.output[0]
                                            for next_node in graph.node:
                                                if next_output_name in next_node.input and (next_node.op_type == 'Conv' or node.op_type=="QLinearConv"):
                                                    next_kernel_shape = None
                                                    for attr in next_node.attribute:
                                                        if attr.name == 'kernel_shape':
                                                            next_kernel_shape = attr.ints
                                                            break
                                                    if next_kernel_shape == [1, 1]:
                                                        has_parallel_conv_sequence = True
                                                        break

            # Break out of the loop if a parallel convolution sequence is found
            if has_parallel_conv_sequence:
                break

    # Print the result
    if has_parallel_conv_sequence:
        print("The model has the parallel convolutional layer sequence: 1x1, 3x3, 5x5, 1x1")
    else:
        print("The model does not have the parallel convolutional layer sequence: 1x1, 3x3, 5x5, 1x1")
